In [ ]:
import numpy as np
import os
import pylab
import seaborn
import scipy.stats as sp
import pickle
import imageio
from scipy.spatial import distance
import json
from collections import OrderedDict
import os.path as osp
import pandas as pd
import copy
import scipy.stats as ss
import sys
import matplotlib.pyplot as plt
DIV_LINE_WIDTH = 50
print(np.__version__)
print(sys.executable)

In [ ]:
def get_all_runs(logdir, condition=None, minlen=20):
    #print(logdir)
    """
    Recursively look through logdir for output files produced by
    Assumes that any file "progress.txt" is a valid hit. 
    """
    datasets = []
    for root, _, files in os.walk(logdir):
        print(root)
        if 'config.json' in files:
            run_name = root[13:]
            exp_name = None
            try:
                config_path = open(os.path.join(root,'config.json'))
                config = json.load(config_path)
                if 'exp_name' in config:
                    exp_name = config['exp_name']
                    
            except:
                print('No file named config.json')
            
            data_dict = {}
            data_dict['config'] = config
            #print(config)
            
            #if nb_epochs >= minlen:
            if True:
                if 'env_params_save.pkl' in files:
                    try:
                        env_params_dict = pickle.load( open(os.path.join(root,'env_params_save.pkl'), "rb" ) )
                    except EOFError:
                        print('Corrupted save, ignoring {}'.format(data_dict['config']['seed']))
                        continue
                    for k,v in env_params_dict.items():
                        if k == 'tasks_alps':
                            data_dict[k] = None
                        elif k == 'tasks_origin':
                            data_dict[k] = None
                        else:
                            data_dict[k] = v
#                           if k == 'epochs_score' or k == 'epochs_episode_nb':
#                               data_dict[k] = v
            
                        

                #print(data_dict.keys())
                datasets.append(data_dict)
#                 if not "epochs_score" in data_dict:
#                     print('ERRRRRRRRRRRRRRRROR {}'.format(run_name))
#                 else:
                print('{} -> {}'.format(run_name, len(data_dict['epochs_score'])))
    return datasets

def get_datasets(rootdir):
    print('loading all experiments located in {}'.format(rootdir))
    _, models_list, _ = next(os.walk(rootdir))
    print(models_list)
    for dir_name in models_list.copy():
        if "ignore" in dir_name:
            models_list.remove(dir_name)
    for expe_name in list(labels.keys()):
        if expe_name not in models_list:
            del labels[expe_name]
    
#     # setting specific colors for each expe
#     for i,m_name in enumerate(models_list):
#         if m_name in specific_colors:
#             colors[m_name] = specific_colors[m_name]
#         else:
#             colors[m_name] = default_colors[i]
            
    # setting per-model type colors    
    for i,m_name in enumerate(models_list):
        for m_type, m_color in per_model_colors.items():
            if m_type in m_name:
                colors[m_name] = m_color
        print("extracting data for {}...".format(m_name))
        m_id = m_name
        models_saves[m_id] = OrderedDict()
        models_saves[m_id]['data'] = get_all_runs(rootdir+m_name)
        print("done")
        if m_name not in labels:
            labels[m_name] = m_name

    """
    retrieve all experiences located in "data to vizu" folder
    """
default_colors = ["violet","yellow",'green','black',u'#ff7f0e',
                      "cyan", "pink", u'#1f77b4',"grey","r",
                     "darkorchid","sienna","lightpink","blue","magenta", "indigo","mediumseagreen",'aqua',
                'deeppink','silver','khaki','goldenrod','y','y','y','y','y','y','y','y','y','y','y','y' ]  + ['y']*50
print(len(default_colors))
labels = OrderedDict()
per_model_colors = OrderedDict()

# LOAD DATA
models_saves = OrderedDict()
colors = OrderedDict()

get_datasets("teachDRL/data/") #

if per_model_colors:  # order runs for legend order as in per_models_colors, with corresponding colors
    ordered_labels = OrderedDict()
    for teacher_type in per_model_colors.keys():
        for k,v in labels.items():
            if teacher_type in k:
                ordered_labels[k] = v
    labels = ordered_labels
else:
    print('not using per_model_color')
    for k in models_saves.keys():
        labels[k] = k
        
print(labels)
print(colors)

In [ ]:
#################################  BUILD CLASSROOM  #############################################
m_id = "YOUR EXP NAME HERE"
is_v2 = False
if 'v2' in m_id:
    is_v2 = True

# Construct history of trained students
def load_expert_trajectory(data, alp_thr=0.2):
    # 1 - loading the trajectory
    for k, v in data.items():
        if k == 'means' or k == 'covariances' or k == 'weights' or k == 'env_train_len' or k == 'episodes' \
           or k == 'env_train_rewards':
            data[k] = v
        if k == 'egt_means' or k == 'egt_covariances' or k == 'egt_weights' or k == 'egt_env_train_len' or k == 'egt_episodes' \
                or k == 'egt_env_train_rewards':
            data[k[4:]] = v

    # 2 - pre-processing expert trajectory
    # removing low-alp gaussians
    processed_gmms_means = []
    processed_gmms_covs = []
    processed_gmms_mean_rew = []
    idx_removed_gmms = []
    gmm_step = data['episodes'][0]
    max_lp = 0.0
    for i, (gmm_means, gmm_covs, episode) in enumerate(zip(data["means"], data["covariances"], data['episodes'])):
        #step_nb += sum(data['env_train_len'][i*gmm_step:(i+1)*gmm_step])
        processed_gmm_means = []
        processed_gmm_covs = []
        all_rewards = data['env_train_rewards'][episode:episode + gmm_step] # from gmm
        rewards = all_rewards[-50:]  # consider mean reward after some training on the GMM
        mean_reward = np.mean(rewards)
        for j, (means, covs) in enumerate(zip(gmm_means, gmm_covs)):
            if means[-1] > max_lp:
                max_lp = means[-1]
            if means[-1] > alp_thr:  # last mean is ALP dimension
                # add gaussian
                processed_gmm_means.append(means)
                processed_gmm_covs.append(covs)
        if not processed_gmm_means == []:  # gmm not empty after pre-process, lets add it
            processed_gmms_means.append(processed_gmm_means)
            processed_gmms_covs.append(processed_gmm_covs)
            processed_gmms_mean_rew.append(mean_reward)
        else:
            idx_removed_gmms.append(i)
    print('idx of removed gmms ({}/{}) in expert traj: {}, max_lp={}'.format(len(data['means']) - len(processed_gmms_means),
                                                                  len(data['means']),
                                                                  idx_removed_gmms, max_lp))
    #print('number of steps: {}'.format(step_nb))
    return len(processed_gmms_means)


folder = 'teachDRL/graphics/elders/raw/'

ignore_failed_trainings = True # SET TO TRUE WHEN BUILDING CLASSROOM
student_ids = []
if is_v2:
    student_params = {'start_cube_idx':[]}
else:
    student_params = {'nb_rot':[]}
nb_possible_pretrain_epochs = 10
initial_test_vectors_list = [[] for _ in range(nb_possible_pretrain_epochs)] # lists of first test vectors from idx 0 to 10
last_test_vector = []
last_perfs = []
max_lps = []


print(m_id)
runs_data = models_saves[m_id]['data']
#print(len(runs_data))
all_alpgmm_tasks_percentage = []
nb_zero_perf = 0
nb_zero_perf_small_net = 0
nb_small_net = 0
for r,run in enumerate(runs_data):
    test_env_vectors = []
    last_perf = round(run["epochs_score"][-1],1)
    max_perf = round(max(run["epochs_score"]),1)
    perf_str = "last:{}, max:{}".format(last_perf, max_perf)
    stud_param_name = 'start_cube_idx' if is_v2 else 'nb_rot'
    stud_p = run['config'][stud_param_name]
    if last_perf == 0.0:
        nb_zero_perf += 1
        #max_lps.append(load_expert_trajectory(run, alp_thr=0.2))
        #print(run['config']['ac_kwargs']['hidden_sizes'])
    perf_title = "Last perf: {}% | Max perf: {}%".format(last_perf, max_perf)
    initial_score = run['epochs_score'][1]  # index 0 is comp at 0 episodes
    last_score = run['epochs_score'][-1]
    
    #if not is_quad and leg_s < 0.3:
    print('seed: {}, param:{}, last_perf: {} score={}-->{}: imp:{}'.format(run['config']['seed'],
                                                               stud_p,
                                                               last_perf,
                                                               initial_score,last_score,
                                                                last_score >= initial_score))

    if (last_score >= initial_score and load_expert_trajectory(run, alp_thr=0.2) > 0) or not ignore_failed_trainings: # use in history only if the student managed to progress and has high-lp gaussians
        last_perfs.append(last_perf)
        student_ids.append(m_id + '_s' + str(run['config']['seed']))
        ep_comp_grids = run['epochs_comp_grid'][1:]  # IMPORTANT: ignore first 0ep perf (always 0)
        for idx, ep_comp_grid in enumerate(ep_comp_grids):
            if idx < nb_possible_pretrain_epochs:
                initial_test_vectors_list[idx].append(ep_comp_grid.flatten())
            if idx == len(ep_comp_grids)-1:
                last_test_vector.append(ep_comp_grid.flatten())
        
        student_params[stud_param_name].append(stud_p)
        #student_params['is_quad'].append(is_quad)
    else:
        print('do not use {}: no progress'.format(m_id))
         
            
                
print('{}/128 failed learning'.format(nb_zero_perf))
#print('{}/{} failed learning'.format(nb_zero_perf_small_net,nb_small_net))
print(max_lps)
print(len(initial_test_vectors_list[0]))
if ignore_failed_trainings:
    
    pickle.dump([student_ids, initial_test_vectors_list, last_test_vector, last_perfs, student_params],
                # WAAAAAAAAAAAAAAAAAAAAAAAAAARNING OTHER
                open( "teachDRL/data/elders_knowledge/toy_classroom.pkl", "wb" ))